In [13]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Database connection settings
db_user = "postgres"
db_password = "Mahlatsi#0310"
db_host = "localhost"   # since you mapped 5432:5432
db_port = "5432"
db_name = "campus_safety"

# Create engine
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

In [6]:
query = "SELECT * FROM safety_reports;"
safety_reports = pd.read_sql(query, engine)
safety_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   report_id    213 non-null    int64         
 1   reporter     213 non-null    object        
 2   category     213 non-null    object        
 3   description  213 non-null    object        
 4   lat          213 non-null    float64       
 5   lon          213 non-null    float64       
 6   timestamp    213 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 11.8+ KB


In [10]:
query = "SELECT * FROM safe_routes;"
safe_routes = pd.read_sql(query, engine)
safe_routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   route_id     213 non-null    int64         
 1   origin       213 non-null    object        
 2   destination  213 non-null    object        
 3   risk_score   213 non-null    float64       
 4   timestamp    213 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 8.4+ KB


In [8]:
query = "SELECT * FROM loadshedding;"
loadshedding = pd.read_sql(query, engine)
loadshedding.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          213 non-null    int64         
 1   area        213 non-null    object        
 2   stage       213 non-null    int64         
 3   start_time  213 non-null    datetime64[ns]
 4   end_time    213 non-null    datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 8.4+ KB


In [12]:
# Convert timestamps to datetime
safety_reports['timestamp'] = pd.to_datetime(safety_reports['timestamp'])
safe_routes['timestamp'] = pd.to_datetime(safe_routes['timestamp'])
loadshedding['start_time'] = pd.to_datetime(loadshedding['start_time'])
loadshedding['end_time'] = pd.to_datetime(loadshedding['end_time'])

In [21]:
fig = px.scatter_mapbox(
    safety_reports,
    lat="lat", lon="lon",
    color="category",
    hover_name="reporter",
    hover_data=["description", "timestamp"],
    zoom=12.9,
    height=600
)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(title="Safety Reports by Location")
fig.show()

C:\Users\Admin\AppData\Local\Temp\ipykernel_13368\2473125773.py:1: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [22]:
fig = px.bar(
    safety_reports['category'].value_counts().reset_index(),
    x='index', y='category',
    labels={'index': 'Category', 'category': 'Count'},
    title="Crime Reports by Category"
)
fig.show()


ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['category', 'count'] but received: index
 To use the index, pass it in directly as `df.index`.

In [ ]:
import plotly.express as px

# Count categories
category_counts = safety_reports['category'].value_counts().reset_index()
category_counts.columns = ['category', 'count']

# Define severity mapping (you can edit this as needed)
severity_colors = {     
    "harassment": "yellow",
    "accident": "yellow",  
    "theft": "red",        
    "suspicious_activity": "green",
    "other": "gray"       
}

# Apply colors
category_counts["color"] = category_counts["category"].map(severity_colors).fillna("lightgray")

# Create pie chart
fig = px.pie(
    category_counts,
    names="category",
    values="count",
    title="Distribution of Safety Report Categories (by Severity)",
    hole=0.35,  # donut style
    color="category",
    color_discrete_map=severity_colors
)

fig.update_traces(textinfo="percent+label")

fig.show()


In [25]:
reports_per_day = safety_reports.groupby(safety_reports['timestamp'].dt.date).size().reset_index(name='counts')

fig = px.line(
    reports_per_day,
    x='timestamp', y='counts',
    title="Trend of Safety Reports Over Time"
)
fig.show()

In [ ]:
import requests
import plotly.graph_objects as go
import ast  # to safely evaluate strings like "(lat, lon)"

# Convert origin and destination columns into tuples
safe_routes['origin_coords'] = safe_routes['origin'].apply(lambda x: ast.literal_eval(x))
safe_routes['destination_coords'] = safe_routes['destination'].apply(lambda x: ast.literal_eval(x))

def get_route_osrm(start, end):
    """
    start, end: tuples (lat, lon)
    returns: list of coordinates along roads
    """
    url = f"http://router.project-osrm.org/route/v1/driving/{start[1]},{start[0]};{end[1]},{end[0]}?overview=full&geometries=geojson"
    resp = requests.get(url).json()
    coords = resp['routes'][0]['geometry']['coordinates']
    # OSRM returns [lon, lat], convert to [lat, lon] for Plotly
    return [(lat, lon) for lon, lat in coords]

# Color mapping function
def risk_to_color(risk):
    if risk >= 0.75:
        return "red"
    elif risk >= 0.4:
        return "yellow"
    else:
        return "green"

fig = go.Figure()

# Assuming you have origin_coords and destination_coords columns as (lat, lon) tuples
for _, row in safe_routes.iterrows():
    path = get_route_osrm(row['origin_coords'], row['destination_coords'])
    lats, lons = zip(*path)

    fig.add_trace(go.Scattermapbox(
        lat=lats,
        lon=lons,
        mode="lines",
        line=dict(width=3, color=risk_to_color(row['risk_score'])),
        hovertext=f"Origin: {row['origin']}<br>"
                  f"Destination: {row['destination']}<br>"
                  f"Risk: {row['risk_score']:.2f}<br>"
                  f"Time: {row['timestamp']}",
        hoverinfo="text",
        showlegend=False  # ensures no legend appears
    ))

fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        zoom=13,
        center=dict(
            lat=safe_routes['origin_coords'].apply(lambda x: x[0]).mean(),
            lon=safe_routes['origin_coords'].apply(lambda x: x[1]).mean()
        )
    ),
    height=600,
    title="Safe vs Risky Routes on Roads"
)

fig.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_13368\4037933318.py:31: DeprecationWarning:

*scattermapbox* is deprecated! Use *scattermap* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [37]:
fig = px.histogram(
    safe_routes,
    x="risk_score",
    nbins=30,
    title="Distribution of Route Risk Scores"
)
fig.show()

In [28]:
fig = px.timeline(
    loadshedding,
    x_start="start_time",
    x_end="end_time",
    y="area",
    color="stage",
    title="Loadshedding Timeline per Area"
)

fig.update_yaxes(autorange="reversed")
fig.show()


In [29]:
# Count safety reports per hour
reports_per_hour = safety_reports.groupby(safety_reports['timestamp'].dt.floor('H')).size().reset_index(name='reports')

# Expand loadshedding into hours
loadshedding_expanded = []
for _, row in loadshedding.iterrows():
    rng = pd.date_range(row['start_time'], row['end_time'], freq='H')
    for t in rng:
        load_shedding_point = {
            "timestamp": t,
            "area": row['area'],
            "stage": row['stage']
        }
        loadshedding_expanded.append(load_shedding_point)

loadshedding_df = pd.DataFrame(loadshedding_expanded)

# Merge by timestamp
merged = pd.merge(reports_per_hour, loadshedding_df, on="timestamp", how="inner")

# Plot correlation
fig = px.scatter(
    merged,
    x="stage", y="reports",
    color="area",
    title="Correlation Between Loadshedding Stage & Safety Reports"
)
fig.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_13368\1075149529.py:2: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

C:\Users\Admin\AppData\Local\Temp\ipykernel_13368\1075149529.py:7: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [30]:
fig = px.density_mapbox(
    safety_reports, 
    lat='lat', 
    lon='lon', 
    z=None,  # can use "severity" if you have a numeric value
    radius=15,  # smaller radius = more detail
    hover_data=['reporter', 'category', 'description'],
    center={'lat': safety_reports['lat'].mean(), 'lon': safety_reports['lon'].mean()},
    zoom=12.9,
    height=600
)

fig.update_layout(mapbox_style="open-street-map", title="Safety Reports Heatmap")
fig.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_13368\1984855508.py:1: DeprecationWarning:

*density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

